# APPLICAZIONE MODELLO A FVG

**Importo tutte le librerie**

In [1]:
import numpy as np                                  # libreria matematica
import scipy as sp
import matplotlib.pyplot as plt                     # librerie grafiche
import matplotlib.colors as mcolors
import pandas as pd                                 # estrazione e lettura dati

**Definisco tutti i parametri fissi**

In [2]:
NEQ = 5         #numero equazioni
n = 4           #numero città

# fissiamo i parametri
beta = 1.12                  #days^-1
mu = 0.55
alfa = 0.14
Z = 3.69                     #days
D = 3.47                     #days
teta = 1.0


#MATRICE DEL PENDOLARISMO

#Legenda:    Trieste = 1, Udine = 2, Gorizia = 3, Pordenone = 4

M = np.zeros((n + 1, n + 1))
Mfvg = pd.read_csv('Mfvg.txt', names=['0','1','2','3','4'])
print(Mfvg)
for i in range(1,n + 1):
    c = str(i)
    M[i,:] = Mfvg.query(c)


#REPRODUCTION NUMBER
    
R0 = alfa*beta*D + (1-alfa)*mu*beta*D
print('R0=',R0)


     0        1       2       3       4
0  0.0     0.00    0.00    0.00    0.00
1  0.0     0.00  665.98  955.46   84.88
2  0.0   700.76    0.00  399.18  376.14
3  0.0  1416.38  885.28    0.00   25.32
4  0.0   136.32  759.64   31.54    0.00
R0= 2.3823632000000003


**FUNZIONE CAMPO VETTORIALE**

In [3]:
#funzione campo vettoriale per RK4

def f(x, K, N, n, NEQ) :
    
    Xeff = np.zeros((n + 1, NEQ + 1)) 
    Neff = np.zeros(n + 1)
    
    
##################### VARIABILI EFFICACI ############################################

    for i in range(1, n + 1):
        
        U1  = np.zeros(1)
        U2  = np.zeros(1)
        U3  = np.zeros(1)
        U4  = np.zeros(1)
        U5  = np.zeros(1)
        U6  = np.zeros(1)        # Spazio allocato per tutte le variabili
        U7  = np.zeros(1)
        U8  = np.zeros(1)
        U9  = np.zeros(1)
        U10 = np.zeros(1)
        U11 = np.zeros(1)
        U12 = np.zeros(1)

        U3s  = np.zeros(NEQ + 1)
        U4s  = np.zeros(NEQ + 1)
        U7s  = np.zeros(NEQ + 1)
        U8s  = np.zeros(NEQ + 1)
        U11s = np.zeros(NEQ + 1)
        U12s = np.zeros(NEQ + 1)
        

        for j in range(1, n + 1):
            
            U3s[j]  = (M[i][j]*X[j][1]) /(N[j] - X[j][3])
            U4s[j]  = (M[j][i]*X[i][1]) /(N[i] - X[i][3])
            U7s[j]  = (M[i][j]*X[j][2]) /(N[j] - X[j][3]) 
            U8s[j]  = (M[j][i]*X[i][2]) /(N[i] - X[i][3])        # per  i termini contenenti una sommatoria costruisco dei
            U11s[j] = (M[i][j]*X[j][4]) /(N[j] - X[j][3])        # vettori che hanno per componenti i termini delle sommatorie
            U12s[j] = (M[j][i]*X[i][4]) /(N[i] - X[i][3])
            
        U3  = teta*sum(U3s)
        U4  = teta*sum(U4s)
        U7  = teta*sum(U7s)                                     # dopodichè li sommo
        U8  = teta*sum(U8s)
        U11 = teta*sum(U11s)
        U12 = teta*sum(U12s)
        
        U3 = max(U3,0)
        U4 = max(U4,0)
        U7 = max(U7,0)
        U8 = max(U8,0)
        U11 = max(U11,0)
        U12 = max(U12,0)

        U3r  = np.random.poisson(U3,1)
        U4r  = np.random.poisson(U4,1)
        U7r  = np.random.poisson(U7,1)
        U8r  = np.random.poisson(U8,1)
        U11r = np.random.poisson(U11,1)
        U12r = np.random.poisson(U12,1)
        
        Xeff[i][1] = X[i][1] - U4r
        Xeff[i][2] = X[i][2] + U7r - U8r
        Xeff[i][3] = X[i][3]                       # i casi riconosciuti non viaggiano!!!!
        Xeff[i][4] = X[i][4] + U11r - U12r
        Neff[i] = Xeff[i][1] +  Xeff[i][2] + Xeff[i][3] + Xeff[i][4] + X[i][5]
        
        
############################### EQUAZIONI #################################################
    
    for i in range(1, n + 1):
        
        Uvar = np.zeros(n + 1)
        
        for j in range(1, n + 1):
            
            Uvar[j] = ((M[i][j]*X[i][1])/(N[i]-X[i][3])) * beta * (( Xeff[j][2] + mu*Xeff[j][4])/ Neff[j])

        Uvar = teta*sum(Uvar) 
        Uvar = max(Uvar,0)
        Uvarr = np.random.poisson(Uvar,1)
        
        U1  = (beta*Xeff[i][1]*Xeff[i][3])/Neff[i]
        U2  = (mu*beta*Xeff[i][1]*Xeff[i][4])/Neff[i]
        U5  = alfa*X[i][2] / Z
        U6  = (1 - alfa)* X[i][2] / Z                          # costruisco gli U rimanenti
        U9  = X[i][3] / D
        U10 = X[i][4] / D
        
        
#############################   STOCASTICO POISSON  #########################################################
        
        U1 = max(U1,0)
        U2 = max(U2,0)
        U5 = max(U5,0)
        U6 = max(U6,0)
        U9 = max(U9,0)
        U10 = max(U10,0)
        
        
        U1r  = np.random.poisson(U1,1)
        U2r  = np.random.poisson(U2,1)
        U5r  = np.random.poisson(U5,1)
        U6r  = np.random.poisson(U6,1)
        U9r  = np.random.poisson(U9,1)
        U10r = np.random.poisson(U10,1)
        
        
###################################### SOMMO GLI U PER FORMARE LE EQUAZIONI ############################################
        
        K[i][1] = - U1r - U2r - Uvarr
        K[i][2] = U1r + U2r - U5r - U6r + Uvarr
        K[i][3] = U5r - U9r
        K[i][4] = U6r - U10r 
        K[i][5] = U9r + U10r
        
    return K, N

**FUNZIONE RK4**

In [4]:
# h è il passo di integrazione #

h = 1  #Integrazione con passo 1 giorno

########################################    FUNZIONE RK4    ###################################################################

def RK4(X, N, h, n, NEQ):
    
    X2 = np.zeros((n + 1, NEQ + 1))                 #spazio allocato
    k1 = np.zeros((n + 1, NEQ + 1))
    k2 = np.zeros((n + 1, NEQ + 1))
    k3 = np.zeros((n + 1, NEQ + 1))
    k4 = np.zeros((n + 1, NEQ + 1))
    
##  STEP 1
        
    for i in range(1, n + 1):
        for j in range(1, NEQ + 1):
            X2[i][j] = X[i][j]                 #copio in X2 i dati iniziali
            
    f(X2, k1, N, n, NEQ)
    
##  STEP 2
        
    for i in range(1, n + 1):                   
        for j in range(1, NEQ + 1):
            X2[i][j] = X[i][j] + k1[i][j] * h/2                     
    f(X2, k2, N, n, NEQ)
    
##  STEP 3
        
    for i in range(1, n + 1):
        for j in range(1, NEQ + 1):
            X2[i][j] = X[i][j] + k2[i][j] * h/2           
    f(X2, k3, N, n, NEQ)
    
##  STEP 4

    for i in range(1, n + 1):
        for j in range(1, NEQ + 1):
            X2[i][j] = X[i][j] + k2[i][j] * h/2           
    f(X2, k4, N, n, NEQ)

##  STEP 5
    
    
    for i in range(1, n + 1):
        for j in range(1, NEQ + 1):
            X[i][j] =  X[i][j] + np.round( h * ( k1[i][j] /6 +  k2[i][j] / 3 +  k3[i][j] /3 + k4[i][j] /6) )  
            
    return

**DATI INIZIALI E NUMERO DI INTEGRAZIONI**

In [5]:
#INTERVALLO INTEGRAZIONE

ndays = 200
t = np.linspace(0, ndays -1 , ndays)

#NUMERO DI INTEGRAZIONI

nrip = 1000


#matrice che contiene tutti i risultati di integrazione stocastici

S  = np.zeros((nrip, n + 1, len(t)))  
E  = np.zeros((nrip, n + 1, len(t)))
Ir = np.zeros((nrip, n + 1, len(t)))
Iu = np.zeros((nrip, n + 1, len(t)))
R  = np.zeros((nrip, n + 1, len(t)))


for i in range(0,nrip):
    
    np.random.seed(i)
    print('ripetizione n°',i)
    
    X = np.zeros((n + 1, NEQ + 2))       #LEGENDA:  S = [][1] ; E = [][2] ; Ir = [][3] ; Iu = [][4]
    N = np.zeros(n + 1)
    
#---------------------  DATI INIZIALI  -------------------------------------------------------------------------------------------

    X[1][1] = 202123.     # S(TS)
    X[1][4] = 1.          # Iu(TS)
    X[2][1] = 98287.      # S(UD)
    X[3][1] = 35212.      # S(GO)
    X[4][1] = 50583.      # S(PN)
     
    
    for z in range(1, n + 1):
        N[z] = sum(X[z,:])
   
    for j in range(1, n + 1):

        S[i][j][0]  = X[j][1]
        E[i][j][0]  = X[j][2]
        Ir[i][j][0] = X[j][3]
        Iu[i][j][0] = X[j][4]
        R[i][j][0]  = X[j][5]
        
#----------------------------------  INTEGRAZIONE  ---------------------------------------------------------------------------------
    
    for r in range(1,len(t)):   
        
        RK4(X, N, h, n, NEQ)
        
        #print(sum(X[4,:]))   #+ sum(X[2,:]) + sum(X[3,:]) + sum(X[4,:]))   
        
              
        for j in range(1, n + 1):
            
            S[i][j][r]  = X[j][1]
            E[i][j][r]  = X[j][2]
            Ir[i][j][r] = X[j][3]
            Iu[i][j][r] = X[j][4]
            R[i][j][r]  = X[j][5]
            
        


ripetizione n° 0
ripetizione n° 1
ripetizione n° 2
ripetizione n° 3
ripetizione n° 4
ripetizione n° 5
ripetizione n° 6
ripetizione n° 7
ripetizione n° 8
ripetizione n° 9
ripetizione n° 10
ripetizione n° 11
ripetizione n° 12
ripetizione n° 13
ripetizione n° 14
ripetizione n° 15
ripetizione n° 16
ripetizione n° 17
ripetizione n° 18
ripetizione n° 19
ripetizione n° 20
ripetizione n° 21
ripetizione n° 22
ripetizione n° 23
ripetizione n° 24
ripetizione n° 25
ripetizione n° 26
ripetizione n° 27
ripetizione n° 28
ripetizione n° 29
ripetizione n° 30
ripetizione n° 31
ripetizione n° 32
ripetizione n° 33
ripetizione n° 34
ripetizione n° 35
ripetizione n° 36
ripetizione n° 37
ripetizione n° 38
ripetizione n° 39
ripetizione n° 40
ripetizione n° 41
ripetizione n° 42
ripetizione n° 43
ripetizione n° 44
ripetizione n° 45
ripetizione n° 46
ripetizione n° 47
ripetizione n° 48
ripetizione n° 49
ripetizione n° 50
ripetizione n° 51
ripetizione n° 52
ripetizione n° 53
ripetizione n° 54
ripetizione n° 55
ri

ripetizione n° 437
ripetizione n° 438
ripetizione n° 439
ripetizione n° 440
ripetizione n° 441
ripetizione n° 442
ripetizione n° 443
ripetizione n° 444
ripetizione n° 445
ripetizione n° 446
ripetizione n° 447
ripetizione n° 448
ripetizione n° 449
ripetizione n° 450
ripetizione n° 451
ripetizione n° 452
ripetizione n° 453
ripetizione n° 454
ripetizione n° 455
ripetizione n° 456
ripetizione n° 457
ripetizione n° 458
ripetizione n° 459
ripetizione n° 460
ripetizione n° 461
ripetizione n° 462
ripetizione n° 463
ripetizione n° 464
ripetizione n° 465
ripetizione n° 466
ripetizione n° 467
ripetizione n° 468
ripetizione n° 469
ripetizione n° 470
ripetizione n° 471
ripetizione n° 472
ripetizione n° 473
ripetizione n° 474
ripetizione n° 475
ripetizione n° 476
ripetizione n° 477
ripetizione n° 478
ripetizione n° 479
ripetizione n° 480
ripetizione n° 481
ripetizione n° 482
ripetizione n° 483
ripetizione n° 484
ripetizione n° 485
ripetizione n° 486
ripetizione n° 487
ripetizione n° 488
ripetizione 

ripetizione n° 869
ripetizione n° 870
ripetizione n° 871
ripetizione n° 872
ripetizione n° 873
ripetizione n° 874
ripetizione n° 875
ripetizione n° 876
ripetizione n° 877
ripetizione n° 878
ripetizione n° 879
ripetizione n° 880
ripetizione n° 881
ripetizione n° 882
ripetizione n° 883
ripetizione n° 884
ripetizione n° 885
ripetizione n° 886
ripetizione n° 887
ripetizione n° 888
ripetizione n° 889
ripetizione n° 890
ripetizione n° 891
ripetizione n° 892
ripetizione n° 893
ripetizione n° 894
ripetizione n° 895
ripetizione n° 896
ripetizione n° 897
ripetizione n° 898
ripetizione n° 899
ripetizione n° 900
ripetizione n° 901
ripetizione n° 902
ripetizione n° 903
ripetizione n° 904
ripetizione n° 905
ripetizione n° 906
ripetizione n° 907
ripetizione n° 908
ripetizione n° 909
ripetizione n° 910
ripetizione n° 911
ripetizione n° 912
ripetizione n° 913
ripetizione n° 914
ripetizione n° 915
ripetizione n° 916
ripetizione n° 917
ripetizione n° 918
ripetizione n° 919
ripetizione n° 920
ripetizione 

### facciamo le statistiche città per città

In [6]:
safe = np.zeros(n)

for i in range(0,nrip):
    for j in range(0,n ):
        if S[i,j+1,ndays-1] > (2./3.)*S[i,j+1,0]:
            safe[j] = safe[j] + 1

numeroripetizioni = str(nrip)

città = ['Trieste', 'Udine', 'Gorizia', 'Pordenone']

for i in range(0,n):
    print('epidemie evitate in ' + città[i] + '=', safe[i], '/' + numeroripetizioni)
    


epidemie evitate in Trieste= 252.0 /1000
epidemie evitate in Udine= 252.0 /1000
epidemie evitate in Gorizia= 252.0 /1000
epidemie evitate in Pordenone= 252.0 /1000


## Salvo i dati con pandas

In [7]:
S_TS = pd.DataFrame(S[:,1,:])
E_TS = pd.DataFrame(E[:,1,:])
Ir_TS = pd.DataFrame(Ir[:,1,:])
Iu_TS = pd.DataFrame(Iu[:,1,:])
R_TS = pd.DataFrame(R[:,1,:])
np.savetxt('S_TS ',S_TS )
np.savetxt('E_TS ',E_TS )
np.savetxt('Ir_TS ',Ir_TS )
np.savetxt('Iu_TS ',Iu_TS )
np.savetxt('R_TS ',R_TS )

S_UD = pd.DataFrame(S[:,2,:])
E_UD = pd.DataFrame(E[:,2,:])
Ir_UD = pd.DataFrame(Ir[:,2,:])
Iu_UD = pd.DataFrame(Iu[:,2,:])
R_UD = pd.DataFrame(R[:,2,:])
np.savetxt('S_UD ',S_UD )
np.savetxt('E_UD ',E_UD )
np.savetxt('Ir_UD ',Ir_UD )
np.savetxt('Iu_UD ',Iu_UD )
np.savetxt('R_UD ',R_UD )

S_GO = pd.DataFrame(S[:,3,:])
E_GO = pd.DataFrame(E[:,3,:])
Ir_GO = pd.DataFrame(Ir[:,3,:])
Iu_GO = pd.DataFrame(Iu[:,3,:])
R_GO = pd.DataFrame(R[:,3,:])
np.savetxt('S_GO ',S_GO )
np.savetxt('E_GO ',E_GO )
np.savetxt('Ir_GO ',Ir_GO )
np.savetxt('Iu_GO ',Iu_GO )
np.savetxt('R_GO ',R_GO )

S_PN = pd.DataFrame(S[:,4,:])
E_PN = pd.DataFrame(E[:,4,:])
Ir_PN= pd.DataFrame(Ir[:,4,:])
Iu_PN = pd.DataFrame(Iu[:,4,:])
R_PN = pd.DataFrame(R[:,4,:])
np.savetxt('S_PN ',S_PN )
np.savetxt('E_PN ',E_PN )
np.savetxt('Ir_PN ',Ir_PN )
np.savetxt('Iu_PN ',Iu_PN )
np.savetxt('R_PN ',R_PN )

temp = pd.DataFrame(t)
np.savetxt('tempi',temp)

stat = pd.DataFrame(safe/nrip)
np.savetxt('statistiche',stat)    